# Demo: sample crawl

In [1]:
import pandas as pd
from selenium import webdriver
import time
from selenium.webdriver.chrome.service import Service
import unidecode
from urllib.parse import unquote

PATH = Service("C:\Program Files (x86)\chromedriver.exe")

driver = webdriver.Chrome(service = PATH)
driver.close()

In [2]:
whoscored_url=[
    "https://1xbet.whoscored.com/Regions/252/Tournaments/2/England-Premier-League",
    "https://1xbet.whoscored.com/Regions/108/Tournaments/5/Italy-Serie-A",
    "https://1xbet.whoscored.com/Regions/206/Tournaments/4/Spain-LaLiga",
    "https://1xbet.whoscored.com/Regions/81/Tournaments/3/Germany-Bundesliga",
    "https://1xbet.whoscored.com/Regions/74/Tournaments/22/France-Ligue-1"
]

In [3]:
PATH = Service("C:\Program Files (x86)\chromedriver.exe")

driver = webdriver.Chrome(service = PATH)
driver.get(whoscored_url[0])

teams_url_raw = \
driver.find_elements("xpath", "(//table[@class='grid with-centered-columns hover']/tbody/tr/td/a[@class='team-link '])")
teams_url = []

for team_url_raw in teams_url_raw:
    teams_url.append(team_url_raw.get_attribute('href'))
driver.close()
teams_url

['https://1xbet.whoscored.com/Teams/167/Show/England-Manchester-City',
 'https://1xbet.whoscored.com/Teams/26/Show/England-Liverpool',
 'https://1xbet.whoscored.com/Teams/15/Show/England-Chelsea',
 'https://1xbet.whoscored.com/Teams/30/Show/England-Tottenham',
 'https://1xbet.whoscored.com/Teams/13/Show/England-Arsenal',
 'https://1xbet.whoscored.com/Teams/32/Show/England-Manchester-United',
 'https://1xbet.whoscored.com/Teams/29/Show/England-West-Ham',
 'https://1xbet.whoscored.com/Teams/14/Show/England-Leicester',
 'https://1xbet.whoscored.com/Teams/211/Show/England-Brighton',
 'https://1xbet.whoscored.com/Teams/161/Show/England-Wolverhampton',
 'https://1xbet.whoscored.com/Teams/23/Show/England-Newcastle',
 'https://1xbet.whoscored.com/Teams/162/Show/England-Crystal-Palace',
 'https://1xbet.whoscored.com/Teams/189/Show/England-Brentford',
 'https://1xbet.whoscored.com/Teams/24/Show/England-Aston-Villa',
 'https://1xbet.whoscored.com/Teams/18/Show/England-Southampton',
 'https://1xbe

In [4]:
PATH = Service("C:\Program Files (x86)\chromedriver.exe")

driver = webdriver.Chrome(service = PATH)
driver.get(teams_url[1])

players_url_raw = driver.find_elements("xpath","//tbody[@id='player-table-statistics-body']/tr/td/a[@class='player-link']")
players_url = []

for player_url_raw in players_url_raw:
    players_url.append(unidecode.unidecode(unquote(player_url_raw.get_attribute('href').lower().replace('show','history'))))
driver.close()
players_url

KeyboardInterrupt: 

## tabular extract

In [ ]:
linkz = players_url[0]
PATH = Service("C:\Program Files (x86)\chromedriver.exe")

driver = webdriver.Chrome(service = PATH)
driver.get(linkz)

#table structure
cols = driver.find_elements('xpath',"//table[@id='top-player-stats-summary-grid']/thead/tr/th")[2:]

for i in range(len(cols)):
    cols[i] = cols[i].get_attribute('innerHTML')
    

data =  driver.find_elements('xpath',"//tbody[@id='player-table-statistics-body']/tr")

pd_appender = []
for dt in data[:-1]:
    dtnr = {'name':linkz[linkz.find('history/')+8:]}
    dtnr.update(zip(cols,dt.text.split()))
    pd_appender.append(dtnr)

driver.close()


In [ ]:
e_table = pd.DataFrame(pd_appender)
e_table

## apply data type for columns & grouping

In [ ]:
e_table['Season'] = e_table['Season'].apply(lambda ss:ss[:4])
e_table['Apps'] = e_table['Apps'].apply\
(lambda apps: int(apps[:apps.find('(')])+int(apps[apps.find('(')+1:apps.find(')')]) if not apps.isdigit() else int(apps))
e_table['Mins'] = e_table['Mins'] .apply(lambda s: int(s) if s.isdigit() else 0)
e_table['Goals'] = e_table['Goals'] .apply(lambda s: int(s) if s.isdigit() else 0)
e_table['Assists'] = e_table['Assists'] .apply(lambda s: int(s) if s.isdigit() else 0)
e_table['Yel'] = e_table['Yel'] .apply(lambda s: int(s) if s.isdigit() else 0)
e_table['MotM'] = e_table['MotM'] .apply(lambda s: int(s) if s.isdigit() else 0)
e_table['Red'] = e_table['Red'] .apply(lambda s: int(s) if s.isdigit() else 0)
e_table['Rating'] = e_table['Rating'] .apply(lambda s: float(s) if s != '-' else 0)

In [ ]:
e_table

In [ ]:
e_table = e_table.iloc[:,[0,1,4,5,6,7,8,9,-2,-1]]
e_table

In [ ]:
def func_agg(x):
    d = []
    for col in (list(e_table)[2:-1]):
        d.append(x[col].sum())
    d.append((x['Rating']*x['Apps']).sum()/(x['Apps'].sum()))
    return pd.Series(d, index = list(e_table)[2:])

In [ ]:
test = e_table.groupby(["name","Season"],as_index = False).apply(func_agg)
for col in list(test)[2:-1]:
    test[col] = test[col].astype('int16')
    print (col, test[col].dtype)
test['Rating'] = test['Rating'].apply(lambda rt: float("%.2f" %rt))
test

# Final crawl

In [7]:
whoscored_url=[
    "https://1xbet.whoscored.com/Regions/252/Tournaments/2/England-Premier-League",
    "https://1xbet.whoscored.com/Regions/108/Tournaments/5/Italy-Serie-A",
    "https://1xbet.whoscored.com/Regions/206/Tournaments/4/Spain-LaLiga",
    "https://1xbet.whoscored.com/Regions/81/Tournaments/3/Germany-Bundesliga",
    "https://1xbet.whoscored.com/Regions/74/Tournaments/22/France-Ligue-1"
]

In [8]:
PATH = Service("C:\Program Files (x86)\chromedriver.exe")

driver = webdriver.Chrome(service = PATH)

teams_url = []

for url in whoscored_url:
    driver.get(url)

    teams_url_raw =\
    driver.find_elements("xpath", "(//table[@class='grid with-centered-columns hover']/tbody/tr/td/a[@class='team-link '])")


    for team_url_raw in teams_url_raw:
        teams_url.append(team_url_raw.get_attribute('href'))
driver.close()
len(teams_url)

98

We wont touch some of the next codebox again, since we stored the data we needed

In [11]:
PATH = Service("C:\Program Files (x86)\chromedriver.exe")

driver = webdriver.Chrome(service = PATH)
players_url = []
for team_url in teams_url:
    driver.get(team_url)
    #time.sleep(3)
    players_url_raw = driver.find_elements("xpath","//tbody[@id='player-table-statistics-body']/tr/td/a[@class='player-link']")

    for player_url_raw in players_url_raw:
        #rlink = 
        #if rlink not in players_url:
        players_url.append(unidecode.unidecode(unquote(player_url_raw.get_attribute('href').lower().replace('show','history'))))

driver.close()
len(players_url)


2926

In [15]:
len(set(players_url))

2795

In [12]:
'''
f = open("player_url.txt","w")
for url in players_url:
    f.write(url+"\n")
f.close()
print(sum(1 for line in open('player_url.txt')))
'''

2926


So in total data of ~2800 players will be collected

In [3]:
f = open("player_url.txt","r")
players_url = f.read().split()
f.close()
players_url[:5]

['https://1xbet.whoscored.com/players/73084/history/kevin-de-bruyne',
 'https://1xbet.whoscored.com/players/303139/history/rodri',
 'https://1xbet.whoscored.com/players/128967/history/joao-cancelo',
 'https://1xbet.whoscored.com/players/331254/history/phil-foden',
 'https://1xbet.whoscored.com/players/136741/history/bernardo-silva']

Get table structure

In [4]:
PATH = Service("C:\Program Files (x86)\chromedriver.exe")

driver = webdriver.Chrome(service = PATH)
pd_appender = []
#table structure
linkz = players_url[0]
driver.get(linkz)


cols = driver.find_elements('xpath',"//table[@id='top-player-stats-summary-grid']/thead/tr/th")[2:]

for i in range(len(cols)):
    cols[i] = cols[i].get_attribute('innerHTML')
driver.close()
cols

['Season',
 'Team',
 'Tournament',
 'Apps',
 'Mins',
 'Goals',
 'Assists',
 'Yel',
 'Red',
 'SpG',
 'PS%',
 'AerialsWon',
 'MotM',
 'Rating']

Now begins the crawling part, we only do 300 players at once, to avoid driver crash

In [160]:
#crawl
PATH = Service("C:\Program Files (x86)\chromedriver.exe")

driver = webdriver.Chrome(service = PATH)
pd_appender = []
for linkz in players_url:
    driver.get(linkz)
    
    data =  driver.find_elements('xpath',"//tbody[@id='player-table-statistics-body']/tr")

    for dt in data[:-1]:
        dtnr = {'name':linkz[linkz.find('history/')+8:]}
        apd = []
        a = dt.text.split('\n')
        apd.append(a[0][:dt.text.find(' ')])
        apd.append(a[0][dt.text.find(' ')+1:dt.text.find('\n')])
        apd += a[1].split()
        dtnr.update(zip(cols, apd))
        pd_appender.append(dtnr)
driver.close()

Next codebox: initializing, run only once

In [120]:
'''e_table = pd.DataFrame(pd_appender)
e_table'''

'e_table = pd.DataFrame(pd_appender)\ne_table'

From here we will repeat

In [161]:
e_table = pd.concat([e_table, pd.DataFrame(pd_appender)])
e_table

,name,Season,Team,Tournament,Apps,Mins,Goals,Assists,Yel,Red,SpG,PS%,AerialsWon,MotM,Rating
0,kevin-de-bruyne,2021/2022,Belgium,WCQ,4,353,2,2,-,-,2.5,82.2,1.5,2,8.12
1,kevin-de-bruyne,2022/2023,Belgium,UNL,3,237,1,-,-,-,3,85.5,-,-,7.28
2,kevin-de-bruyne,2021/2022,Man City,EFLC,2,173,1,-,-,-,4,82.9,0.5,-,7.98
3,kevin-de-bruyne,2021/2022,Man City,UCL,9(1),735,2,3,2,-,2.4,85.5,0.2,1,7.15
4,kevin-de-bruyne,2021/2022,Man City,EPL,25(5),2205,15,8,2,-,2.6,82.5,0.4,4,7.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6923,sebastian-vasiliadis,2019/2020,Paderborn,GB,32,2811,1,1,10,-,1.2,76,0.7,-,6.62
6924,sebastian-vasiliadis,2018/2019,Paderborn,GB2,23(5),2076,6,10,4,-,1.4,77.8,1,1,7.11
6925,fabian-kunze,2021/2022,Arminia Bielefeld,GB,13(9),1260,-,-,9,-,0.5,75.7,1.7,-,6.40
6926,fabian-kunze,2020/2021,Arminia Bielefeld,GB,12(14),1261,-,-,6,-,0.3,76,1.5,-,6.32


In [162]:
e_table.drop_duplicates(keep = 'first', inplace = True)
e_table

,name,Season,Team,Tournament,Apps,Mins,Goals,Assists,Yel,Red,SpG,PS%,AerialsWon,MotM,Rating
0,kevin-de-bruyne,2021/2022,Belgium,WCQ,4,353,2,2,-,-,2.5,82.2,1.5,2,8.12
1,kevin-de-bruyne,2022/2023,Belgium,UNL,3,237,1,-,-,-,3,85.5,-,-,7.28
2,kevin-de-bruyne,2021/2022,Man City,EFLC,2,173,1,-,-,-,4,82.9,0.5,-,7.98
3,kevin-de-bruyne,2021/2022,Man City,UCL,9(1),735,2,3,2,-,2.4,85.5,0.2,1,7.15
4,kevin-de-bruyne,2021/2022,Man City,EPL,25(5),2205,15,8,2,-,2.6,82.5,0.4,4,7.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26,stephane-bahoken,2017/2018,Strasbourg,FL1,19(7),1666,7,1,5,-,1.4,74.8,1.1,-,6.45
27,stephane-bahoken,2012/2013,Nice,FL1,3(7),270,2,-,1,-,0.5,76.3,0.2,-,6.28
28,stephane-bahoken,2011/2012,Nice,FL1,0(5),68,-,-,1,-,0.4,71.4,0.2,-,6.14
29,stephane-bahoken,2010/2011,Nice,FL1,0(1),11,-,-,-,-,1,-,-,-,6.12


In [164]:
e_table.reset_index(drop = True, inplace = True)
e_table

,name,Season,Team,Tournament,Apps,Mins,Goals,Assists,Yel,Red,SpG,PS%,AerialsWon,MotM,Rating
0,kevin-de-bruyne,2021/2022,Belgium,WCQ,4,353,2,2,-,-,2.5,82.2,1.5,2,8.12
1,kevin-de-bruyne,2022/2023,Belgium,UNL,3,237,1,-,-,-,3,85.5,-,-,7.28
2,kevin-de-bruyne,2021/2022,Man City,EFLC,2,173,1,-,-,-,4,82.9,0.5,-,7.98
3,kevin-de-bruyne,2021/2022,Man City,UCL,9(1),735,2,3,2,-,2.4,85.5,0.2,1,7.15
4,kevin-de-bruyne,2021/2022,Man City,EPL,25(5),2205,15,8,2,-,2.6,82.5,0.4,4,7.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26159,stephane-bahoken,2017/2018,Strasbourg,FL1,19(7),1666,7,1,5,-,1.4,74.8,1.1,-,6.45
26160,stephane-bahoken,2012/2013,Nice,FL1,3(7),270,2,-,1,-,0.5,76.3,0.2,-,6.28
26161,stephane-bahoken,2011/2012,Nice,FL1,0(5),68,-,-,1,-,0.4,71.4,0.2,-,6.14
26162,stephane-bahoken,2010/2011,Nice,FL1,0(1),11,-,-,-,-,1,-,-,-,6.12


In [165]:
e_table['Season'] = e_table['Season'].apply(lambda ss:ss[:4])
e_table['Apps'] = e_table['Apps'].apply\
(lambda apps: int(apps[:apps.find('(')])+int(apps[apps.find('(')+1:apps.find(')')]) if not apps.isdigit() else int(apps))
e_table['Mins'] = e_table['Mins'] .apply(lambda s: int(s) if s.isdigit() else 0)
e_table['Goals'] = e_table['Goals'] .apply(lambda s: int(s) if s.isdigit() else 0)
e_table['Assists'] = e_table['Assists'] .apply(lambda s: int(s) if s.isdigit() else 0)
e_table['Yel'] = e_table['Yel'] .apply(lambda s: int(s) if s.isdigit() else 0)
e_table['MotM'] = e_table['MotM'] .apply(lambda s: int(s) if s.isdigit() else 0)
e_table['Red'] = e_table['Red'] .apply(lambda s: int(s) if s.isdigit() else 0)
e_table['Rating'] = e_table['Rating'] .apply(lambda s: float(s) if s != '-' else 0)

e_table

,name,Season,Team,Tournament,Apps,Mins,Goals,Assists,Yel,Red,SpG,PS%,AerialsWon,MotM,Rating
0,kevin-de-bruyne,2021,Belgium,WCQ,4,353,2,2,0,0,2.5,82.2,1.5,2,8.12
1,kevin-de-bruyne,2022,Belgium,UNL,3,237,1,0,0,0,3,85.5,-,0,7.28
2,kevin-de-bruyne,2021,Man City,EFLC,2,173,1,0,0,0,4,82.9,0.5,0,7.98
3,kevin-de-bruyne,2021,Man City,UCL,10,735,2,3,2,0,2.4,85.5,0.2,1,7.15
4,kevin-de-bruyne,2021,Man City,EPL,30,2205,15,8,2,0,2.6,82.5,0.4,4,7.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26159,stephane-bahoken,2017,Strasbourg,FL1,26,1666,7,1,5,0,1.4,74.8,1.1,0,6.45
26160,stephane-bahoken,2012,Nice,FL1,10,270,2,0,1,0,0.5,76.3,0.2,0,6.28
26161,stephane-bahoken,2011,Nice,FL1,5,68,0,0,1,0,0.4,71.4,0.2,0,6.14
26162,stephane-bahoken,2010,Nice,FL1,1,11,0,0,0,0,1,-,-,0,6.12


In [166]:
e_table = e_table.iloc[:,[0,1,4,5,6,7,8,9,-2,-1]]
e_table

,name,Season,Apps,Mins,Goals,Assists,Yel,Red,MotM,Rating
0,kevin-de-bruyne,2021,4,353,2,2,0,0,2,8.12
1,kevin-de-bruyne,2022,3,237,1,0,0,0,0,7.28
2,kevin-de-bruyne,2021,2,173,1,0,0,0,0,7.98
3,kevin-de-bruyne,2021,10,735,2,3,2,0,1,7.15
4,kevin-de-bruyne,2021,30,2205,15,8,2,0,4,7.64
...,...,...,...,...,...,...,...,...,...,...
26159,stephane-bahoken,2017,26,1666,7,1,5,0,0,6.45
26160,stephane-bahoken,2012,10,270,2,0,1,0,0,6.28
26161,stephane-bahoken,2011,5,68,0,0,1,0,0,6.14
26162,stephane-bahoken,2010,1,11,0,0,0,0,0,6.12


In [167]:
def func_agg(x):
    d = []
    for col in (list(e_table)[2:-1]):
        d.append(x[col].sum())
    d.append((x['Rating']*x['Apps']).sum()/(x['Apps'].sum()))
    return pd.Series(d, index = list(e_table)[2:])

In [168]:
final_table = e_table.groupby(["name","Season"],as_index = False).apply(func_agg)
for col in list(e_table)[2:-1]:
    final_table[col] = final_table[col].astype('int16')
final_table['Rating'] = final_table['Rating'].apply(lambda rt: float("%.2f" %rt))
final_table

,name,Season,Apps,Mins,Goals,Assists,Yel,Red,MotM,Rating
0,Adam-szalai,2009,15,791,1,1,0,0,0,6.41
1,Adam-szalai,2010,20,1287,4,3,1,0,1,6.82
2,Adam-szalai,2011,15,1072,3,1,2,1,0,6.66
3,Adam-szalai,2012,29,2406,13,3,6,0,3,6.97
4,Adam-szalai,2013,35,1850,7,0,2,0,1,6.58
...,...,...,...,...,...,...,...,...,...,...
16130,zlatan-ibrahimovic,2017,6,119,0,0,0,0,0,6.17
16131,zlatan-ibrahimovic,2018,27,2143,22,6,5,1,8,7.84
16132,zlatan-ibrahimovic,2019,49,4155,41,13,9,0,16,7.82
16133,zlatan-ibrahimovic,2020,24,1742,15,3,3,1,8,7.39


In [169]:
final_table["name"] = final_table["name"].apply(lambda ss: ss.lower())
final_table

,name,Season,Apps,Mins,Goals,Assists,Yel,Red,MotM,Rating
0,adam-szalai,2009,15,791,1,1,0,0,0,6.41
1,adam-szalai,2010,20,1287,4,3,1,0,1,6.82
2,adam-szalai,2011,15,1072,3,1,2,1,0,6.66
3,adam-szalai,2012,29,2406,13,3,6,0,3,6.97
4,adam-szalai,2013,35,1850,7,0,2,0,1,6.58
...,...,...,...,...,...,...,...,...,...,...
16130,zlatan-ibrahimovic,2017,6,119,0,0,0,0,0,6.17
16131,zlatan-ibrahimovic,2018,27,2143,22,6,5,1,8,7.84
16132,zlatan-ibrahimovic,2019,49,4155,41,13,9,0,16,7.82
16133,zlatan-ibrahimovic,2020,24,1742,15,3,3,1,8,7.39


In [170]:
final_table.name.describe()

count              16135
unique              2789
top       seamus-coleman
freq                  14
Name: name, dtype: object

In [172]:
final_table.to_csv("player_performace.csv",index=False)